In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor, Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_ace_24 = pd.read_csv("Ace_погружение_24часа.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_24 = pd.read_csv("Discover_погружение_24часа_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_ace_autocorr = pd.read_csv("Ace_погружение_автокорр.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_autocorr = pd.read_csv("Discover_погружение_автокорр_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])

In [3]:
datasets = [data_ace_24, data_discover_24, data_ace_autocorr, data_discover_autocorr]

In [4]:
  for dataset in datasets:
    dataset.set_index('datetime', inplace=True)
    dataset.dropna(inplace=True)
    if not dataset.index.is_monotonic_increasing:
        dataset = dataset.sort_index(inplace=True)

#### Разбиения данных на тренировочные и тестовые наборы
Обозначения:\
x..._1_24 — горизонт предсказания 1 час, погружение 24 часа на все переменные\
x..._3_a — горизонт предсказания 3 часа, погружение с учётом автокорреляционной функции

In [5]:
from_index_data = 4 #индекс, с которого начинаются данные о величине ММП и параметрах СВ

In [6]:
x_train_ace_3_24, x_test_ace_3_24, y_train_ace_3_24, y_test_ace_3_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +3'], test_size=0.15, shuffle=False)
x_train_ace_2_24, x_test_ace_2_24, y_train_ace_2_24, y_test_ace_2_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +2'], test_size=0.15, shuffle=False)
x_train_ace_1_24, x_test_ace_1_24, y_train_ace_1_24, y_test_ace_1_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +1'], test_size=0.15, shuffle=False)

In [7]:
x_train_disc_3_24, x_test_disc_3_24, y_train_disc_3_24, y_test_disc_3_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +3'], test_size=0.15, shuffle=False)
x_train_disc_2_24, x_test_disc_2_24, y_train_disc_2_24, y_test_disc_2_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +2'], test_size=0.15, shuffle=False)
x_train_disc_1_24, x_test_disc_1_24, y_train_disc_1_24, y_test_disc_1_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +1'], test_size=0.15, shuffle=False)

In [8]:
x_train_ace_3_a, x_test_ace_3_a, y_train_ace_3_a, y_test_ace_3_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +3'], test_size=0.15, shuffle=False)
x_train_ace_2_a, x_test_ace_2_a, y_train_ace_2_a, y_test_ace_2_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +2'], test_size=0.15, shuffle=False)
x_train_ace_1_a, x_test_ace_1_a, y_train_ace_1_a, y_test_ace_1_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +1'], test_size=0.15, shuffle=False)

In [9]:
x_train_disc_3_a, x_test_disc_3_a, y_train_disc_3_a, y_test_disc_3_a = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +3'], test_size=0.15, shuffle=False)
x_train_disc_2_a, x_test_disc_2_a, y_train_disc_2_a, y_test_disc_2_a = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +2'], test_size=0.15, shuffle=False)
x_train_disc_1_a, x_test_disc_1_a, y_train_disc_1_a, y_test_disc_1_a = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +1'], test_size=0.15, shuffle=False)

In [10]:
model_cat = CatBoostRegressor(iterations=100,
                              depth=2,
                              learning_rate=1,
                              loss_function='RMSE',
                              logging_level='Silent')

pipe = make_pipeline(StandardScaler(), LinearRegression())

### Обучение на данных ACE, тестирование на данных ACE

#### Погружение - 24 часа по всем переменным

In [11]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_24, x_test_ace_1_24, y_train_ace_1_24, y_test_ace_1_24 #прогноз на 1 час
y_train_2, y_test_2 = y_train_ace_2_24, y_test_ace_2_24 #прогноз на 2 часа
y_train_3, y_test_3 = y_train_ace_3_24, y_test_ace_3_24 #прогноз на 3 часа

In [12]:
for data in [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]:
    data = data.to_numpy().astype(float)

In [13]:
for i in range(1, 4):
  pipe = make_pipeline(StandardScaler(), LinearRegression())
  pipe.fit(x_train, eval(f'y_train_{i}'))
  y_pred = pipe.predict(x_test)

  R_2 = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
  MSE = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
  MAE = metrics.mean_absolute_error(eval(f'y_test_{i}'), y_pred)

  print(f'Прогноз на {i} час:')
  print(f'MSE={MSE} и r^2={R_2} при использовании линейной регрессии, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=380.14129275412785 и r^2=-1.9047364117136243 при использовании линейной регрессии, погружение - 24 часа по всем переменным
Прогноз на 2 час:
MSE=343.4238862273051 и r^2=-1.624360846350167 при использовании линейной регрессии, погружение - 24 часа по всем переменным
Прогноз на 3 час:
MSE=262.0423883515334 и r^2=-1.001825499982183 при использовании линейной регрессии, погружение - 24 часа по всем переменным


In [14]:
lambda_values = [0.01, 10, 1000, 100000, 500000, 1000000, 5000000, 10000000]

for i in range(1, 4):
    for lambda_val in lambda_values:
        ridge_reg = Ridge(lambda_val)
        ridge_reg.fit(x_train, eval(f'y_train_{i}'))
        y_pred = ridge_reg.predict(x_test)
        mse_ridge = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
        r2_ridge = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
        print(f'Прогноз на {i} час:')
        print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=380.1412804605016 и r^2=-1.9047363177755425 с Lambda=0.01 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=380.1289996992608 и r^2=-1.904642477998019 с Lambda=10 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=378.91779370020225 и r^2=-1.8953874082789106 с Lambda=1000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=294.02911877207686 и r^2=-1.246735894471036 с Lambda=100000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=164.18402225389656 и r^2=-0.2545632814769099 с Lambda=500000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=109.10316142994232 и r^2=0.16632070317180336 с Lambda=1000000 при испо

In [15]:
for i in range(1, 4):
    train_pool = Pool(x_train, eval(f'y_train_{i}'), cat_features=[])
    test_pool = Pool(x_test, eval(f'y_test_{i}'), cat_features=[])

    model_cat.fit(train_pool)
    y_pred_cat = model_cat.predict(test_pool)

    R_2_catboost = metrics.r2_score(eval(f'y_test_{i}'), y_pred_cat)
    MSE_catboost = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred_cat)

    print(f'Прогноз на {i} час:')
    print(f'MSE={MSE_catboost} и r^2={R_2_catboost} при использовании модели CatBoost, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=17.780049073499466 и r^2=0.8641390532144757 при использовании модели CatBoost, погружение - 24 часа по всем переменным
Прогноз на 2 час:
MSE=26.271281761213462 и r^2=0.7992413282752022 при использовании модели CatBoost, погружение - 24 часа по всем переменным
Прогноз на 3 час:
MSE=34.56520860231303 и r^2=0.7359453314877833 при использовании модели CatBoost, погружение - 24 часа по всем переменным


#### Погружение с учётом автокорреляционной функции

In [16]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_a, x_test_ace_1_a, y_train_ace_1_a, y_test_ace_1_a #прогноз на 1 час
y_train_2, y_test_2 = y_train_ace_2_a, y_test_ace_2_a #прогноз на 2 часа
y_train_3, y_test_3 = y_train_ace_3_a, y_test_ace_3_a #прогноз на 3 часа

In [17]:
for data in [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]:
    data = data.to_numpy().astype(float)

In [18]:
for i in range(1, 4):
  pipe = make_pipeline(StandardScaler(), LinearRegression())
  pipe.fit(x_train, eval(f'y_train_{i}'))
  y_pred = pipe.predict(x_test)

  R_2 = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
  MSE = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
  MAE = metrics.mean_absolute_error(eval(f'y_test_{i}'), y_pred)

  print(f'Прогноз на {i} час:')
  print(f'MSE={MSE} и r^2={R_2} при использовании линейной регрессии, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=383.13181431534736 и r^2=-1.9388516170404464 при использовании линейной регрессии, погружение с учётом автокорреляционной функции
Прогноз на 2 час:
MSE=347.63587482929825 и r^2=-1.667512323123368 при использовании линейной регрессии, погружение с учётом автокорреляционной функции
Прогноз на 3 час:
MSE=265.78144211913514 и r^2=-1.0385084046212758 при использовании линейной регрессии, погружение с учётом автокорреляционной функции


In [19]:
lambda_values = [0.01, 10, 1000, 100000, 500000, 1000000, 5000000, 10000000]

for i in range(1, 4):
    for lambda_val in lambda_values:
        ridge_reg = Ridge(lambda_val)
        ridge_reg.fit(x_train, eval(f'y_train_{i}'))
        y_pred = ridge_reg.predict(x_test)
        mse_ridge = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
        r2_ridge = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
        print(f'Прогноз на {i} час:')
        print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=383.13180184976176 и r^2=-1.9388515214219018 с Lambda=0.01 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=383.1193492660637 и r^2=-1.938756002609916 с Lambda=10 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=381.8910576355366 и r^2=-1.9293342665136262 с Lambda=1000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=295.4993075855245 и r^2=-1.2666575457429028 с Lambda=100000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=163.70927456475192 и r^2=-0.2557486700468641 с Lambda=500000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=108.38254751501299 и r^2=0.16864062

In [20]:
for i in range(1, 4):
    train_pool = Pool(x_train, eval(f'y_train_{i}'), cat_features=[])
    test_pool = Pool(x_test, eval(f'y_test_{i}'), cat_features=[])

    model_cat.fit(train_pool)
    y_pred_cat = model_cat.predict(test_pool)

    R_2_catboost = metrics.r2_score(eval(f'y_test_{i}'), y_pred_cat)
    MSE_catboost = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred_cat)

    print(f'Прогноз на {i} час:')
    print(f'MSE={MSE_catboost} и r^2={R_2_catboost} при использовании модели CatBoost, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=17.351849001531942 и r^2=0.866900874342893 при использовании модели CatBoost, погружение с учётом автокорреляционной функции
Прогноз на 2 час:
MSE=25.564030448071023 и r^2=0.803839674825231 при использовании модели CatBoost, погружение с учётом автокорреляционной функции
Прогноз на 3 час:
MSE=34.17416566756661 и r^2=0.7378886826002407 при использовании модели CatBoost, погружение с учётом автокорреляционной функции


#### Обучение на данных ACE, тестирование на данных DSCOVR

#### Обучение на данных DSCOVR, тестирование на данных ACE

#### Обучение на данных DSCOVR, тестирование на данных DSCOVR

#### Погружение - 24 часа по всем переменным

In [21]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_24, x_test_disc_1_24, y_train_disc_1_24, y_test_disc_1_24 #прогноз на 1 час
y_train_2, y_test_2 = y_train_disc_2_24, y_test_disc_2_24 #прогноз на 2 часа
y_train_3, y_test_3 = y_train_disc_3_24, y_test_disc_3_24 #прогноз на 3 часа

In [22]:
for data in [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]:
    data = data.to_numpy().astype(float)

In [23]:
for i in range(1, 4):
  pipe = make_pipeline(StandardScaler(), LinearRegression())
  pipe.fit(x_train, eval(f'y_train_{i}'))
  y_pred = pipe.predict(x_test)

  R_2 = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
  MSE = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
  MAE = metrics.mean_absolute_error(eval(f'y_test_{i}'), y_pred)

  print(f'Прогноз на {i} час:')
  print(f'MSE={MSE} и r^2={R_2} при использовании линейной регрессии, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=14.35971655043237 и r^2=0.8364879138963685 при использовании линейной регрессии, погружение - 24 часа по всем переменным
Прогноз на 2 час:
MSE=25.791332809132605 и r^2=0.705828815483077 при использовании линейной регрессии, погружение - 24 часа по всем переменным
Прогноз на 3 час:
MSE=29.742611895541927 и r^2=0.6609102306230588 при использовании линейной регрессии, погружение - 24 часа по всем переменным


In [24]:
lambda_values = [0.01, 10, 1000, 100000, 500000, 1000000, 5000000, 10000000]

for i in range(1, 4):
    for lambda_val in lambda_values:
        ridge_reg = Ridge(lambda_val)
        ridge_reg.fit(x_train, eval(f'y_train_{i}'))
        y_pred = ridge_reg.predict(x_test)
        mse_ridge = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
        r2_ridge = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
        print(f'Прогноз на {i} час:')
        print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=14.359716094399884 и r^2=0.8364879190891465 с Lambda=0.01 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=14.359261032483158 и r^2=0.8364931008156138 с Lambda=10 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=14.318515898812452 и r^2=0.8369570599600489 с Lambda=1000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=14.300400905855401 и r^2=0.8371633328539292 с Lambda=100000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=16.58943427598903 и r^2=0.8110984296786569 с Lambda=500000 при использовании линейной регрессии с Ridge-регуляризацией, погружение - 24 часа по всем переменным
Прогноз на 1 час:
MSE=18.656493403532338 и r^2=0.7875611161908094 с Lambda=1000000 при использ

In [25]:
for i in range(1, 4):
    train_pool = Pool(x_train, eval(f'y_train_{i}'), cat_features=[])
    test_pool = Pool(x_test, eval(f'y_test_{i}'), cat_features=[])

    model_cat.fit(train_pool)
    y_pred_cat = model_cat.predict(test_pool)

    R_2_catboost = metrics.r2_score(eval(f'y_test_{i}'), y_pred_cat)
    MSE_catboost = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred_cat)

    print(f'Прогноз на {i} час:')
    print(f'MSE={MSE_catboost} и r^2={R_2_catboost} при использовании модели CatBoost, погружение - 24 часа по всем переменным')

Прогноз на 1 час:
MSE=15.125014385857614 и r^2=0.827773574367347 при использовании модели CatBoost, погружение - 24 часа по всем переменным
Прогноз на 2 час:
MSE=29.749626935260025 и r^2=0.660681242832742 при использовании модели CatBoost, погружение - 24 часа по всем переменным
Прогноз на 3 час:
MSE=33.7899697191418 и r^2=0.6147670863756582 при использовании модели CatBoost, погружение - 24 часа по всем переменным


#### Погружение с учётом автокорреляционной функции

In [26]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_a, x_test_disc_1_a, y_train_disc_1_a, y_test_disc_1_a #прогноз на 1 час
y_train_2, y_test_2 = y_train_disc_2_a, y_test_disc_2_a #прогноз на 2 часа
y_train_3, y_test_3 = y_train_disc_3_a, y_test_disc_3_a #прогноз на 3 часа

In [27]:
for data in [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]:
    data = data.to_numpy().astype(float)

In [28]:
for i in range(1, 4):
  pipe = make_pipeline(StandardScaler(), LinearRegression())
  pipe.fit(x_train, eval(f'y_train_{i}'))
  y_pred = pipe.predict(x_test)

  R_2 = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
  MSE = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
  MAE = metrics.mean_absolute_error(eval(f'y_test_{i}'), y_pred)

  print(f'Прогноз на {i} час:')
  print(f'MSE={MSE} и r^2={R_2} при использовании линейной регрессии, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=14.35971655043237 и r^2=0.8364879138963685 при использовании линейной регрессии, погружение с учётом автокорреляционной функции
Прогноз на 2 час:
MSE=25.791332809132605 и r^2=0.705828815483077 при использовании линейной регрессии, погружение с учётом автокорреляционной функции
Прогноз на 3 час:
MSE=29.742611895541927 и r^2=0.6609102306230588 при использовании линейной регрессии, погружение с учётом автокорреляционной функции


In [29]:
lambda_values = [0.01, 10, 1000, 100000, 500000, 1000000, 5000000, 10000000]

for i in range(1, 4):
    for lambda_val in lambda_values:
        ridge_reg = Ridge(lambda_val)
        ridge_reg.fit(x_train, eval(f'y_train_{i}'))
        y_pred = ridge_reg.predict(x_test)
        mse_ridge = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
        r2_ridge = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
        print(f'Прогноз на {i} час:')
        print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=14.359716094399884 и r^2=0.8364879190891465 с Lambda=0.01 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=14.359261032483158 и r^2=0.8364931008156138 с Lambda=10 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=14.318515898812452 и r^2=0.8369570599600489 с Lambda=1000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=14.300400905855401 и r^2=0.8371633328539292 с Lambda=100000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=16.58943427598903 и r^2=0.8110984296786569 с Lambda=500000 при использовании линейной регрессии с Ridge-регуляризацией, погружение с учётом автокорреляционной функции
Прогноз на 1 час:
MSE=18.656493403532338 и r^2=0.7875611161

In [30]:
for i in range(1, 4):
    train_pool = Pool(x_train, eval(f'y_train_{i}'), cat_features=[])
    test_pool = Pool(x_test, eval(f'y_test_{i}'), cat_features=[])

    model_cat.fit(train_pool)
    y_pred_cat = model_cat.predict(test_pool)

    R_2_catboost = metrics.r2_score(eval(f'y_test_{i}'), y_pred_cat)
    MSE_catboost = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred_cat)

    print(f'Прогноз на {i} час:')
    print(f'MSE={MSE_catboost} и r^2={R_2_catboost} при использовании модели CatBoost, погружение с учётом автокорреляционной функции')

Прогноз на 1 час:
MSE=15.125014385857614 и r^2=0.827773574367347 при использовании модели CatBoost, погружение с учётом автокорреляционной функции
Прогноз на 2 час:
MSE=29.749626935260025 и r^2=0.660681242832742 при использовании модели CatBoost, погружение с учётом автокорреляционной функции
Прогноз на 3 час:
MSE=33.7899697191418 и r^2=0.6147670863756582 при использовании модели CatBoost, погружение с учётом автокорреляционной функции
